In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout, BatchNormalization
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasClassifier
import mysql.connector
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('combined_data.csv')
data.head()


# Features and Target
X = data[['Initial_Year', 'Initial_Month', 'Forecast_Year', 'Foreast_Month', 'Province_ID']]
y = data['Rainfall(mm)']

# Scale the feature data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)



model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  
])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
optimizer = Adam(learning_rate=0.0001, clipvalue=1.0)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])



c:\Users\hoker\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
plt.plot(data)
plt.show()

In [12]:
def predict_rain(model, scaler, input_data):
    # ตรวจสอบว่ามี 4 ฟีเจอร์ ถ้ามี 4 ฟีเจอร์ให้เพิ่ม Province_ID จากข้อมูลที่กรอง
    if len(input_data) == 4:
        last_weather_data = X.iloc[-1][['Initial_Year', 'Initial_Month', 'Forecast_Year', 'Foreast_Month']].values
        input_data = np.concatenate([last_weather_data, [input_data[3]]])  # เพิ่ม Province_ID

    # Normalize ข้อมูล
    input_data_scaled = scaler.transform([input_data])

    # พยากรณ์ด้วยโมเดล
    predict = model.predict(input_data_scaled)
    probability = predict[0][0] * 100  # Convert to percentage
    
    # Return ผลลัพธ์
    if probability > 50:
        return probability, "Rain is likely"
    else:
        return probability, "No Rain expected"

# การเรียกใช้ฟังก์ชัน




print("id เมือง ใช้ในการกรอกพื้นที่ ที่ต้องการ")

provinces = [
    (1, "กรุงเทพมหานคร"),
    (2, "จังหวัดกาญจนบุรี"),
    (3, "จังหวัดการบุรีรัมย์"),
    (4, "จังหวัดขอนแก่น"),
    (5, "จังหวัดจันทบุรี"),
    (6, "จังหวัดชลบุรี"),
    (7, "จังหวัดชุมพร"),
    (8, "จังหวัดเชียงราย"),
    (9, "จังหวัดเชียงใหม่"),
    (10, "จังหวัดตรัง"),
    (11, "จังหวัดตราด"),
    (12, "จังหวัดนครนายก"),
    (13, "จังหวัดนครปฐม"),
    (14, "จังหวัดนครราชสีมา"),
    (15, "จังหวัดนครศรีธรรมราช"),
    (16, "จังหวัดน่าน"),
    (17, "จังหวัดบุรีรัมย์"),
    (18, "จังหวัดปทุมธานี"),
    (19, "จังหวัดพะเยา"),
    (20, "จังหวัดพิษณุโลก"),
    (21, "จังหวัดเพชรบุรี"),
    (22, "จังหวัดเพชรบูรณ์"),
    (23, "จังหวัดแม่ฮ่องสอน"),
    (24, "จังหวัดลำปาง"),
    (25, "จังหวัดลำพูน"),
    (26, "จังหวัดเลย"),
    (27, "จังหวัดศรีสะเกษ"),
    (28, "จังหวัดสกลนคร"),
    (29, "จังหวัดสงขลา"),
    (30, "จังหวัดสุพรรณบุรี"),
    (31, "จังหวัดสุราษฎร์ธานี"),
    (32, "จังหวัดอำนาจเจริญ"),
    (33, "จังหวัดอุดรธานี"),
    (34, "จังหวัดอุตรดิตถ์"),
    (35, "จังหวัดเชียงราย"),
]

for province_id, province_name in provinces:
    print(f"Province ID: {province_id}, Province Name: {province_name}")

print("ตัวอย่างวันที่ 06/09/2024 ใส่ตามที่ขึ้นไม่ใช่ใส่หมดทีเดียว")
iD = int(input("Day: "))
iM = int(input("Month: "))
iY = int(input("Year: "))
city_id = int(input("ใส่เลขเมือง : "))
# ดึงข้อมูลจาก Province_ID
selected_province = next((name for id, name in provinces if id == city_id), None)
if selected_province is None:
    print("ไม่พบจังหวัดที่เลือก")
else:
    print(f"คุณเลือกจังหวัด: {selected_province}")

roun = int(input("Number of training epochs: "))
batch_siz = int(input("Batch size: "))
train_round = int(input("ใส่รอบในการ train ซํ้าเพื่อหาค่าเฉลี่ย แนะนำคือ 3 รอบ ทั่งนี่ หากไม่ต้องการให้ใส่ 1"))
train_iterations = train_round  # กำหนดจำนวนรอบการ train

rain_probabilities = []

for i in range(train_iterations):
    print(f"Training iteration {i+1}/{train_iterations}")
    
    model.fit(X_train, y_train, epochs=roun, batch_size=batch_siz, verbose=0)
    model.save("model.h5")

    # พยากรณ์ด้วยข้อมูลวันสุดท้าย
    X_last_day = X.iloc[-1].values
    probability_last_day, _ = predict_rain(model, scaler, X_last_day[:5])
    
    # เพิ่มค่า rain_probabilities
    rain_probabilities.append(probability_last_day)

average_probability = sum(rain_probabilities) / len(rain_probabilities)

print("----------------------------------------------------------------")
print(f"ค่าเฉลี่ยความน่าจะเป็นในการเกิดฝนหลังจาก {train_iterations} รอบ: {average_probability:.2f}%")
print("================================================================")

result_custom_date = predict_rain(model, scaler, np.array([iD, iM, iY, city_id]))
print("===========================OUTPUT===============================")
print(f"สำหรับวันที่ {iD}/{iM}/{iY}: {result_custom_date[1]} (ความน่าจะเป็น: {result_custom_date[0]:.2f}%)")
print("===================================================================================================")

id เมือง ใช้ในการกรอกพื้นที่ ที่ต้องการ
Province ID: 1, Province Name: กรุงเทพมหานคร
Province ID: 2, Province Name: จังหวัดกาญจนบุรี
Province ID: 3, Province Name: จังหวัดการบุรีรัมย์
Province ID: 4, Province Name: จังหวัดขอนแก่น
Province ID: 5, Province Name: จังหวัดจันทบุรี
Province ID: 6, Province Name: จังหวัดชลบุรี
Province ID: 7, Province Name: จังหวัดชุมพร
Province ID: 8, Province Name: จังหวัดเชียงราย
Province ID: 9, Province Name: จังหวัดเชียงใหม่
Province ID: 10, Province Name: จังหวัดตรัง
Province ID: 11, Province Name: จังหวัดตราด
Province ID: 12, Province Name: จังหวัดนครนายก
Province ID: 13, Province Name: จังหวัดนครปฐม
Province ID: 14, Province Name: จังหวัดนครราชสีมา
Province ID: 15, Province Name: จังหวัดนครศรีธรรมราช
Province ID: 16, Province Name: จังหวัดน่าน
Province ID: 17, Province Name: จังหวัดบุรีรัมย์
Province ID: 18, Province Name: จังหวัดปทุมธานี
Province ID: 19, Province Name: จังหวัดพะเยา
Province ID: 20, Province Name: จังหวัดพิษณุโลก
Province ID: 21, Prov

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


c:\Users\hoker\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Training iteration 2/3


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


c:\Users\hoker\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Training iteration 3/3


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
----------------------------------------------------------------
ค่าเฉลี่ยความน่าจะเป็นในการเกิดฝนหลังจาก 3 รอบ: nan%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
===========================OUTPUT===============================
สำหรับวันที่ 19/9/2024: No Rain expected (ความน่าจะเป็น: nan%)


c:\Users\hoker\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\hoker\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [13]:
history = model.fit(X_scaled, y, epochs=roun, batch_size=batch_siz, verbose=1)
print(history.history['loss'])


Epoch 1/64
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0155 - loss: nan
Epoch 2/64
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0165 - loss: nan
Epoch 3/64
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0150 - loss: nan
Epoch 4/64
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0150 - loss: nan
Epoch 5/64
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0146 - loss: nan
Epoch 6/64
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0146 - loss: nan
Epoch 7/64
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0142 - loss: nan
Epoch 8/64
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0160 - loss: nan  
Epoch 9/64
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0154 - loss: nan
Epoch 10/64
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0148 - loss: nan  
Epoch 11/64
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0135 - loss: nan  
Epoch 12/64
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0168 - loss: